In [11]:
import pandas as pd
import numpy as np
from pathlib import Path

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# https://www.census.gov/topics/education/educational-attainment/data/tables.2010.List_2016040495.html

import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
# https://www.census.gov/topics/education/educational-attainment/data/tables.2010.List_2016040495.html

# Testing

## Data Model

In [9]:
three_year_sample_wght_released = pd.read_csv(Path('../data_for_modeling/2017_2018_2019_sample_wght_released.csv'))

list_value = ['High school graduate',
"Bachelor's degree",
"Some college, no degree",
"Master's degree",
"Associate's degree, academic",
"11th grade2",
"Associate's degree, occupational",
"5th - 6th grade",
"Doctoral degree",
"7th - 8th grade",
"9th grade",
"10th grade",
"Professional degree",
"1st - 4th grade",
"None"]


# Transform Previous Customer column
def changeStatus(status):
    if status == "Doctoral degree":
        return 1
    else:
        return 0

# Along with replace() and map(), this is another way to encode the dr. column into numbers.
three_year_sample_wght_released['Education'] = three_year_sample_wght_released['Education'].apply(changeStatus)

count_doctor = len(three_year_sample_wght_released[three_year_sample_wght_released['Education']==1])
count_not_doctor = len(three_year_sample_wght_released[three_year_sample_wght_released['Education']==0])
pct_of_doctor = count_doctor/(count_doctor+count_not_doctor)
print("percentage of doctor is", pct_of_doctor*100)
pct_of_not_doctor = count_not_doctor/(count_doctor+count_not_doctor)
print("percentage of not doctor", pct_of_not_doctor*100)

# Our classes are imbalanced, and the ratio of dr to no dr instances is 2:98.
#  Before we go ahead to balance the classes, let’s do some more exploration.

data = three_year_sample_wght_released
cat_vars=['Age', 'Race', 'Sex']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1
    
cat_vars=['Age', 'Race', 'Sex']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
data_final=data[to_keep]
data_final.columns.values

data_final

percentage of doctor is 1.611248008333264
percentage of not doctor 98.38875199166674


,Education,Age_18 to 24 years,Age_25 to 29 years,Age_30 to 34 years,Age_35 to 39 years,Age_40 to 44 years,Age_45 to 49 years,Age_50 to 54 years,Age_55 to 59 years,Age_60 to 64 years,Age_65 to 69 years,Age_70 to 74 years,Age_75 years and over,Race_Asian,Race_Black,Race_Hispanic,Race_White,Sex_Female,Sex_Male
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842882,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
842883,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
842884,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
842885,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0


In [10]:
data_final_dropped = data_final.drop(['Age_25 to 29 years','Race_Asian','Sex_Male'], axis=1)
# car_df.drop(['safety_rating', 'passenger_capacity'], axis = 1, inplace = True)
data_final_dropped

,Education,Age_18 to 24 years,Age_30 to 34 years,Age_35 to 39 years,Age_40 to 44 years,Age_45 to 49 years,Age_50 to 54 years,Age_55 to 59 years,Age_60 to 64 years,Age_65 to 69 years,Age_70 to 74 years,Age_75 years and over,Race_Black,Race_Hispanic,Race_White,Sex_Female
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842882,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
842883,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
842884,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
842885,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [13]:
X = data_final_dropped.loc[:, data_final_dropped.columns != 'Education']
y = data_final_dropped.loc[:, data_final_dropped.columns == 'Education']
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['Education'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of not doctors in oversampled data",len(os_data_y[os_data_y['Education']==0]))
print("Number of doctors",len(os_data_y[os_data_y['Education']==1]))
print("Proportion of doctor data in oversampled data is ",len(os_data_y[os_data_y['Education']==0])/len(os_data_X))
print("Proportion of not doctor data in oversampled data is ",len(os_data_y[os_data_y['Education']==1])/len(os_data_X))

length of oversampled data is  1161056
Number of not doctors in oversampled data 580528
Number of doctors 580528
Proportion of doctor data in oversampled data is  0.5
Proportion of not doctor data in oversampled data is  0.5


In [25]:
os_data_X.rename(columns = {'Age_18 to 24 years':'Age_18_to_24_years', 'Age_30 to 34 years':'Age_30_to_34_years',
                            'Age_35 to 39 years':'Age_35_to_39_years', 'Age_40 to 44 years':'Age_40_to_44_years',
                            'Age_45 to 49 years':'Age_45_to_49_years', 'Age_50 to 54 years':'Age_50_to_54_years',
                            'Age_55 to 59 years':'Age_55_to_59_years', 'Age_60 to 64 years':'Age_60_to_64_years',
                            'Age_65 to 69 years':'Age_65_to_69_years', 'Age_70 to 74 years':'Age_70_to_74_years',
                            'Age_75 years and over':'Age_75_years_and_over'}, inplace = True)
os_data_X = os_data_X.assign(DoctorProb=lambda x: x.Age_18_to_24_years * -0.888799 + x.Age_30_to_34_years * 0.095082 + x.Age_35_to_39_years * 0.259210 + x.Age_40_to_44_years * 0.287873 + x.Age_45_to_49_years * 0.409522 +
x.Age_50_to_54_years * 0.326011 + x.Age_55_to_59_years * 0.162633 + x.Age_60_to_64_years * 0.134293 +
x.Age_65_to_69_years * 0.306962 + x.Age_70_to_74_years * 0.394298 + x.Age_75_years_and_over * 0.367415 + 
x.Race_Black * 0.007620 + x.Race_Hispanic * 0.004488 + x.Race_White * -0.111330 + x.Sex_Female * 0.005542)
os_data_X = os_data_X.assign(Education=os_data_y['Education'])


os_data_X

,Age_18_to_24_years,Age_30_to_34_years,Age_35_to_39_years,Age_40_to_44_years,Age_45_to_49_years,Age_50_to_54_years,Age_55_to_59_years,Age_60_to_64_years,Age_65_to_69_years,Age_70_to_74_years,Age_75_years_and_over,Race_Black,Race_Hispanic,Race_White,Sex_Female,Education,DoctorProb
0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.182085
1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0.303734
2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0.022963
3,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0.147880
4,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0.272372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161051,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0.282968
1161052,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0.298192
1161053,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,-0.010706
1161054,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0.108244


In [27]:
os_data_X.to_csv('../data_for_modeling/probability_prediction.csv', index=False)